**Name:** VICENTE, Francheska Josefa

**Section:** DATAPRE N01

# Data Collection

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Bulbapedia
Get the list of the next generations of Pokemons. Make sure you get the following information: (1) kdex, (2) ndex, (3) name, (4) types, and (5) URL to the Pokemon's wiki page.

In [26]:
# source: 02 - Web Scraping With Edits from class
# edited to include the link of Pokemon
URL = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
poke_content = soup.find(id='mw-content-text')
poke_tables = poke_content.find_all('table')

min_generation = 2
max_generation = 9 # there are currently 9 generations of Pokemons

extracted_poke_info = []

for i in range (min_generation, max_generation + 1):
    info_start = 1
    gen_list = poke_tables[i]
    poke_info= gen_list.contents[1]
    info_row = gen_list.contents[info_start]
    
    for pokemon_info_values, even_index_chec in zip(info_row.contents, range(0,len(info_row.contents))):
        if ((even_index_chec % 2) == 0) & (even_index_chec != 0) :
            url = 'https://bulbapedia.bulbagarden.net' + info_row.contents[even_index_chec].find_all('a')[0]['href']
            pokemon_raw_info = pokemon_info_values.text.strip().split('\n')
            generation = i
            if len(pokemon_raw_info) == 7:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = ''
            elif len(pokemon_raw_info) == 8:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = pokemon_raw_info[7]
            elif len(pokemon_raw_info) == 9:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = ''
            elif len(pokemon_raw_info) == 10:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = pokemon_raw_info[9]
            else:
                print('Check out elements containing ' + str(len(pokemon_raw_info)) + ' elements')
        
            extracted_poke_info.append((kdex, ndex, poke_name, type1, type2, generation, url))
            
        else:
            pass

print(extracted_poke_info)

[('#001', '#152', 'Chikorita', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Chikorita_(Pok%C3%A9mon)'), ('#002', '#153', 'Bayleef', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Bayleef_(Pok%C3%A9mon)'), ('#003', '#154', 'Meganium', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Meganium_(Pok%C3%A9mon)'), ('#004', '#155', 'Cyndaquil', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Cyndaquil_(Pok%C3%A9mon)'), ('#005', '#156', 'Quilava', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Quilava_(Pok%C3%A9mon)'), ('#006', '#157', 'Typhlosion', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Typhlosion_(Pok%C3%A9mon)'), ('#157', '', 'Typhlosion', 'Fire', 'Ghost', 2, 'https://bulbapedia.bulbagarden.net/wiki/Typhlosion_(Pok%C3%A9mon)'), ('#007', '#158', 'Totodile', 'Water', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Totodile_(Pok%C3%A9mon)'), ('#008', '#159', 'Croconaw', 'Water', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Croc

In [ ]:
df_extracted_poke_info = pd.DataFrame(extracted_poke_info)
df_extracted_poke_info.columns = ['Kdex', 'Ndex', 'Pokemon', 'Type 1', 'Type 2', 'URL']

df_extracted_poke_info.to_json('Activity 01 - Data Collection (Pokémon).json', orient= 'index')

## News site
Get the news articles published on April 4 - 5. Decide which news site you want to get articles from. Make sure you get the following information: (1) date, (2) title, (3) full article, and (4) author.

In [ ]:
from datetime import datetime
import time

start = 'april 4, 2022'
end = 'april 5, 2022'

START_DATE = datetime.strptime(start, "%B %d, %Y")
END_DATE = datetime.strptime(end, "%B %d, %Y")

checker = True
counter = 1

extracted_articles = []

while checker:
    if counter == 1:
        URL = "https://newsinfo.inquirer.net/source/philippine-daily-inquirer"
    else:
        URL = "https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/" + str(counter)
    
    page = requests.get(URL)
    # print("what page:", URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    news_content = soup.find(id='inq-channel-left')
    # time.sleep(5)
    articles_list = news_content.find_all(id="ch-ls-box")
    
    for i in range (len(articles_list)): 
        article = articles_list[i]
        article_text = article.text.strip().split('\n')
        
        if len(article_text) >= 3:
            date_str = article_text[2].split(' BY:')[0]
            curr_date = datetime.strptime(date_str, "%B %d, %Y")
            
            if curr_date < START_DATE:
                checker = False
            elif START_DATE <= curr_date <= END_DATE:
                article_url = article.find_all('a')[0]['href']
                article_page = requests.get(article_url)
                article_soup = BeautifulSoup(article_page.content, 'html.parser')

                article_title = article_text[1]
                article_content = article_soup.find(id="article_content").text.strip()
                

                if(article_soup.find(id="art_author") is None):
                    article_author = article_soup.find(id="art_plat").find_all("a")[0].text.strip()
                else:
                    article_author = article_soup.find(id="art_author").find_all("a")[0].text.strip()
                    
                extracted_articles.append((article_title, article_author, date_str, article_content))
                
            else:
                continue

    counter = counter + 1
    time.sleep(20)
    
# print("hello", extracted_articles)

what page: https://newsinfo.inquirer.net/source/philippine-daily-inquirer


In [32]:
df_article_list = pd.DataFrame(extracted_articles)
df_article_list.columns = ['Title', 'Author', 'Publication Date', 'Content']
df_article_list
df_article_list.to_json('Activity 01 - Data Collection (News Sites).json', orient= 'index')